# Set Up Env

In [1]:
import sys
import os

import sagemaker
from sagemaker import get_execution_role

# Add the parent directory to the sys.path
sys.path.insert(0, os.path.abspath('..'))

# Define IAM role
role = get_execution_role()
role

# Establish S3 bucket connection
import boto3
s3 = boto3.client('s3')
bucket = 'capstone-bucket-4-friends'

print(os.getcwd())

from file_utilities import s3_download 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
/home/sagemaker-user/capstone-2024-summer/src/jenna


In [2]:
# standard libraries
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
!pip install seaborn -q
import seaborn as sns

In [3]:
#crsp_and_comp = "CRSP/sp500_crsp_compustat_merged_2018_2023.csv"
#crsp_and_comp_path = s3_download(crsp_and_comp)

In [4]:
#index = "Index/security_master.csv"
#index_path = s3_download(index)

In [3]:
crsp_and_comp_df = pd.read_csv("/home/sagemaker-user/capstone-2024-summer/data/sp500_crsp_compustat_merged_2018_2023.csv")

crsp_and_comp_df.head()

/tmp/ipykernel_214/3900309308.py:1: DtypeWarning: Columns (0,21,25,26,27,28,29,30,31,32,33,37,38,39,40,43,44,45,46,47,52,53,54,55,657,658) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_and_comp_df = pd.read_csv("/home/sagemaker-user/capstone-2024-summer/data/sp500_crsp_compustat_merged_2018_2023.csv")


,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,costat,fic,cshtrq,dvpspq,dvpsxq,mkvaltq,prccq,prchq,prclq,adjex
0,68389X10,10104,8045,10536,1,7379,2018-01-02,46.170,47.8011,46.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68389X10,10104,8045,10536,1,7379,2018-01-03,47.440,48.0700,47.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68389X10,10104,8045,10536,1,7379,2018-01-04,47.715,48.1900,48.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,68389X10,10104,8045,10536,1,7379,2018-01-05,48.280,48.6300,48.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68389X10,10104,8045,10536,1,7379,2018-01-08,47.940,49.0700,48.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
crsp_and_comp_df.dtypes

cusip       object
permno       int64
permco       int64
issuno       int64
hexcd        int64
            ...   
mkvaltq    float64
prccq      float64
prchq      float64
prclq      float64
adjex      float64
Length: 667, dtype: object

In [5]:
og_len = len(crsp_and_comp_df)
og_cols = len(crsp_and_comp_df.columns)
print("Number of records:", og_len)
print("Number of columns:", og_cols)

Number of records: 828593
Number of columns: 667


In [6]:
# crsp_and_comp_df.columns.tolist()

# Null Analysis

In [7]:
# Count the number of nulls in each column
null_counts = crsp_and_comp_df.isnull().sum()

# Calculate the percentage of nulls in each column
total_rows = len(crsp_and_comp_df)
null_percentages = (null_counts / total_rows) * 100

In [8]:
null_table = crsp_and_comp_df.isnull().sum().reset_index()
null_table.columns = ["Column", "Null Values"]

# Add column to show the percentage of null values
null_table["Percentage"] = (null_table["Null Values"] / len(crsp_and_comp_df)) * 100

# Sorting the table by the number of null values in descending order
null_table_sorted = null_table.sort_values(by="Null Values", ascending=False).reset_index(drop=True)
null_table_sorted

,Column,Null Values,Percentage
0,ufretsdy,828593,100.0
1,xopty,828593,100.0
2,xoptqpy,828593,100.0
3,xoptepsy,828593,100.0
4,xoptepsqpy,828593,100.0
...,...,...,...
662,permno,0,0.0
663,hsiccd,0,0.0
664,hexcd,0,0.0
665,permco,0,0.0


## A1. Drop all columns which contains more than 80% null as they cannot be reasonably obtained or imputed.

In [9]:
og_cols = len(crsp_and_comp_df.columns)
columns_to_drop = null_percentages[null_percentages > 80].index
crsp_and_comp_df = crsp_and_comp_df.drop(columns=columns_to_drop)
new_cols = len(crsp_and_comp_df.columns)
print("Number of columns removed: ", og_cols - new_cols)
print("Number of columns kept:", new_cols)

Number of columns removed:  310
Number of columns kept: 357


In [10]:
crsp_and_comp_df.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,costat,fic,cshtrq,dvpspq,dvpsxq,mkvaltq,prccq,prchq,prclq,adjex
0,68389X10,10104,8045,10536,1,7379,2018-01-02,46.170,47.8011,46.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68389X10,10104,8045,10536,1,7379,2018-01-03,47.440,48.0700,47.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68389X10,10104,8045,10536,1,7379,2018-01-04,47.715,48.1900,48.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,68389X10,10104,8045,10536,1,7379,2018-01-05,48.280,48.6300,48.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68389X10,10104,8045,10536,1,7379,2018-01-08,47.940,49.0700,48.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def date_formatting(df):
    """Convert to datetime"""
    date_columns = [
        "nameendt",
        "dclrdt",
        "dlpdt",
        "nextdt",
        "paydt",
        "rcrddt",
        "shrenddt",
        "datadate",
        "apdedateq",
        "fdateq",
        "pdateq",
        "rdq",
        "adjex",
        "ipodate",
    ]
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")

date_formatting(crsp_and_comp_df)

## A2. Drop all columns containing duplicate information

In [ ]:
og_cols = len(crsp_df.columns)
columns_to_drop = [
    "permco",       # Dropped CRSP columns that Rachel dropped before
    "issuno",
    "tsymbol",
    "retx",
    "vwretx",
    "ewretx",
    "hexcd",
    "siccd",
    "hsiccd",
    "hsicig",
    "hsicmg",
    "gvkey",        # Used for merging, redundant after join
    "datadate",     # Likely redundant with CRSP's date
    "indfmt",       # Standard industrial format, usually constant
    "consol",       # Consolidation level, usually constant
    "popsrc",       # Population source, usually constant
    "datafmt",      # Data format, usually constant
    "tic",          # Ticker, already in CRSP
    "cusip",        # CUSIP, already in CRSP
    "exchg",        # Exchange code, CRSP has this information
    "prccq",        # Price data, if using CRSP's pricing information
    "cshoq",        # Shares outstanding, if using CRSP's SHROUT
    "ajexq",        # Adjustment factor, if using CRSP's adjustment factors
    "sic",          # SIC code, already in CRSP
    "naics",        # If using CRSP's SIC code instead
]
crsp_and_comp_df = crsp_and_comp_df.drop(columns=columns_to_drop)
new_cols = len(crsp_and_comp_df.columns)
print("Number of columns removed: ", og_cols - new_cols)
print("Number of columns kept:", new_cols)

## Null Analysis Part 2

In [14]:
null_table = crsp_and_comp_df.isnull().sum().reset_index()
null_table.columns = ["Column", "Null Values"]

# Add column to show the percentage of null values
null_table["Percentage"] = (null_table["Null Values"] / len(crsp_and_comp_df)) * 100

# Sorting the table by the number of null values in descending order
null_table_sorted = null_table.sort_values(by="Null Values", ascending=False).reset_index(drop=True)
null_table_sorted

,Column,Null Values,Percentage
0,optvolq,661513,79.835697
1,spidy,658376,79.457104
2,spidq,658123,79.426570
3,optdry,657919,79.401950
4,seteps12,656063,79.177956
...,...,...,...
352,hexcd,0,0.000000
353,permco,0,0.000000
354,permno,0,0.000000
355,shrout,0,0.000000


# Review unique values and distribution in each remaining column

In [15]:
for column in crsp_and_comp_df.columns:
    print("-" * 10, column, "-" * 10)
    print(crsp_and_comp_df[column].nunique())
    print(crsp_and_comp_df[column].value_counts())

---------- cusip ----------
721
cusip
88160R10    1509
68389X10    1509
59491810    1509
74144T10    1509
12503M10    1509
            ... 
56584910      55
82880610      45
66877110      33
72348410      26
88320310      22
Name: count, Length: 721, dtype: int64
---------- permno ----------
568
permno
93436    1509
10104    1509
10107    1509
10138    1509
10145    1509
         ... 
89454     250
23570     249
77668     245
25778     213
89757     205
Name: count, Length: 568, dtype: int64
---------- permco ----------
560
permco
54311    6036
45483    3018
54433    3018
56166    3018
56662    2412
         ... 
43333     250
59446     249
11569     245
5220      213
44118     205
Name: count, Length: 560, dtype: int64
---------- issuno ----------
291
issuno
0        416385
15722      1509
14903      1509
14913      1509
14981      1509
          ...  
38168       250
88473       249
15686       245
24435       213
40191       205
Name: count, Length: 291, dtype: int64
---------- hexc

In [ ]:
company_counts = df['gvkey'].value_counts()
company_counts